In [1]:
import numpy as np
import pandas as pd
import semopy
date = pd.read_excel(r"D:\wechat\WeChat Files\wxid_1lqbllxk3eee22\FileStorage\File\2024-03\473bb4493592e3b9350b974b5fc702ea_492567e8e3f65b0bafe9d75873af93fe_8.xlsx",sheet_name='土壤植被微生物相关',header=1)

In [2]:
wc = date["WC "]
tn = date["TN"]
an = date["AN "]
ph = date["pH "]
xijun = date["Bchao1"]
npp = date["NPP"]
mei = date.iloc[:,11:18]
new_index = {"WC ":"wc","TN":"tn","AN ":"an","pH ":"ph","Bchao1":"xijun","NPP":"npp"}

In [3]:
##将各种酶活性整合为一个指标
#归一化，保证所有酶含量贡献一致
mei = mei.div(mei.mean())
mei =pd.DataFrame(mei.sum(axis=1),columns=["mei"])

In [4]:
date_m = pd.concat([wc,tn,an,ph,xijun,mei,npp], axis=1)

In [5]:
def min_max_normalization(x):
    return (x - x.min()) / (x.max() - x.min())

# 对数据框中的每一列进行最小-最大标准化
date_end = date_m.apply(min_max_normalization)

# 显示标准化后的数据框
# date_end

In [6]:
# 对列进行重命名
date_end = date_end.rename(columns=new_index)

In [7]:
# 定义路径方程
model = '''
# 定义路径方程
an ~ wc
an ~ tn
an ~ ph

ph ~ tn

xijun ~ an
xijun ~ tn

mei ~ xijun
mei ~ tn
mei ~ ph

xijun ~ npp
mei ~ npp
'''

# 拟合模型
mod = semopy.Model(model,mimic_lavaan=True)
fit = mod.fit(data=date_end,clean_slate=True)
# 输出结果
print(fit)

Name of objective: MLW
Optimization method: SLSQP
Optimization successful.
Optimization terminated successfully
Objective value: 0.616
Number of iterations: 30
Params: -0.045 1.030 0.107 -0.673 -0.409 0.326 0.739 0.795 0.963 0.469 -0.584 0.007 0.038 0.043 0.017


In [8]:
mod.inspect(std_est=True)

,lval,op,rval,Estimate,Est. Std,Std. Err,z-value,p-value
0,an,~,wc,-0.045053,-0.059039,0.055001,-0.819132,0.412711
1,an,~,tn,1.029644,1.020914,0.091812,11.214703,0.000000
2,an,~,ph,0.107029,0.114476,0.083285,1.285094,0.198759
3,ph,~,tn,-0.673392,-0.624247,0.172021,-3.914579,0.000091
4,xijun,~,an,-0.408922,-0.528626,0.305730,-1.337530,0.181050
5,xijun,~,tn,0.326224,0.418144,0.314526,1.037192,0.299646
6,xijun,~,npp,0.738506,0.725716,0.161617,4.569483,0.000005
7,mei,~,xijun,0.795427,0.538170,0.292268,2.721571,0.006497
8,mei,~,tn,0.962629,0.834811,0.226537,4.249326,0.000021
9,mei,~,ph,0.468771,0.438532,0.193860,2.418094,0.015602


In [9]:
stats = semopy.calc_stats(mod)
stats.T

,Value
DoF,13.000000
DoF Baseline,24.000000
chi2,14.776562
chi2 p-value,0.321509
chi2 Baseline,120.623839
CFI,0.981614
GFI,0.877499
AGFI,0.773844
NFI,0.877499
TLI,0.966056


In [19]:
date_ = []
with open('mid\土壤-环境因子.txt', 'r') as file:
    # 逐行读取数据
    for line in file:
        date_.append(line.strip().split()) 

zhiwu = pd.DataFrame(date_[1:],columns=date_[0])

# zhiwu     

In [18]:
date_ = []
with open('mid\植物-环境因子.txt', 'r') as file:
    # 逐行读取数据
    for line in file:
        date_.append(line.strip().split()) 

tur = pd.DataFrame(date_[1:],columns=date_[0])

# tur        

In [143]:
date_hj = pd.concat([zhiwu,tur.iloc[:,1:]],axis=1)

In [135]:
date_sample1 = pd.read_excel(r"C:\Users\lll23\Desktop\1.xlsx")
date_sample2 = pd.read_excel(r"C:\Users\lll23\Desktop\2.xlsx")

In [58]:
# date_sample1 = date_sample1.iloc[:,[0,21,22]]
# date_sample2 = date_sample2.iloc[:,[0,21,22]]

In [74]:
# date_sample2.isnull().sum()

In [155]:
date_sample1.columns

Index(['Phylum', 'F10-1', 'F10-2', 'F11-1', 'F11-2', 'F1-1', 'F1-2', 'F12-1',
       'F12-2', 'F2-1', 'F2-2', 'F3-1', 'F3-2', 'F4-1', 'F4-2', 'F5-1', 'F5-2',
       'F6-1', 'F6-2', 'F7-1', 'F7-2', 'F8-1', 'F8-2', 'F9-1', 'F9-2'],
      dtype='object')

In [163]:
# 获得物种的Shannon多样性指数

def normalize_column(col):
    col_sum = col.sum()
    p = col / col_sum
    lnp = np.log(p)*p
    return -p*lnp

# 对除去'Phylum'列之外的所有列应用函数
date_fengdu = date_sample2.iloc[:, 1:].apply(normalize_column).sum()
fd = pd.DataFrame([date_sample1.columns[1:],date_fengdu],index=["sampleId","fd"]).T

c:\Users\lll23\Anaconda3\envs\skl\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [165]:
date_hj.iloc[:,1:] = date_hj.iloc[:,1:].astype(float)

In [167]:
date_all = pd.merge(date_hj,fd, on="sampleId", how='inner')

In [180]:
date_all

,sampleId,SM,SVW,STP,SC,T,SOM,TN,TP,AP,AK,pH,WZFFD,DYXZS,JYDZS,DSSWL,DXSWL,fd
0,F1-1,0.0542,1.16,0.56,4.63,26.6,25.4987,1.5537,0.37,6.705,337.632,8.26,5.0,1.0838,0.6734,118.89,6.46,0.256751
1,F1-2,0.0374,1.25,0.53,4.67,26.3,27.2893,1.6628,0.306,6.524,284.958,8.29,6.0,1.2364,0.6901,134.82,2.8,0.254296
2,F6-1,0.0337,1.33,0.5,7.2,27.6,27.2105,1.6115,0.2987,5.615,236.335,8.77,4.0,0.9,0.6492,99.0,4.63,0.255992
3,F6-2,0.0498,1.13,0.58,7.53,28.1,25.2735,1.5512,0.3202,6.887,228.232,8.81,5.0,1.1431,0.7102,147.31,5.33,0.257485
4,F10-1,0.0375,1.28,0.52,7.97,28.6,27.2701,1.6296,0.2985,5.797,216.076,8.51,4.0,1.0969,0.7912,129.23,1.18,0.236361
5,F10-2,0.0297,1.41,0.47,8.03,28.5,27.799,1.5885,0.285,4.888,203.921,8.58,4.0,1.2824,0.9251,136.96,2.49,0.250774
6,F2-1,0.0482,1.2,0.55,7.7,26.7,30.3732,2.0855,0.2633,4.161,155.298,8.74,4.0,1.0386,0.7492,174.66,3.54,0.243765
7,F2-2,0.0539,1.16,0.56,7.33,27.3,26.9553,2.1246,0.2846,4.888,163.402,8.87,5.0,0.9849,0.6119,52.4,3.74,0.255877
8,F8-1,0.0466,1.49,0.44,7.5,27.7,28.8593,1.6388,0.2847,6.16,179.609,8.77,4.0,1.0698,0.7717,63.53,5.68,0.245499
9,F8-2,0.0441,1.36,0.49,7.73,28.0,30.0016,1.6914,0.2556,5.433,179.609,8.77,3.0,1.0235,0.9317,93.15,4.12,0.253566


In [175]:
####生物丰富度系数和环境各个指标之间的影响
xgx = []
from scipy.stats import pearsonr
for i in range(1,len(date_hj.columns)):
    # 计算皮尔逊相关系数和置信度
    correlation_coefficient, p_value = pearsonr(date_all.iloc[:,i], date_all.iloc[:,-1])
    xgx.append([date_all.columns[i],correlation_coefficient,p_value])

In [181]:
pd.DataFrame(xgx,columns=["1","相关性","皮尔逊系数置信度"])

,1,相关性,皮尔逊系数置信度
0,SM,-0.150722,0.482057
1,SVW,-0.087005,0.686028
2,STP,0.087546,0.684173
3,SC,-0.106283,0.621106
4,T,-0.167064,0.435237
5,SOM,0.104237,0.627876
6,TN,-0.179132,0.402300
7,TP,-0.107261,0.617878
8,AP,-0.195741,0.359327
9,AK,-0.084217,0.695612
